In [1]:
!pip install tensorflow==2.9.0
!pip install tensorflow-text==2.9.0

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import joblib
import time

# 1. 데이터 로드

In [ ]:
# 왓챠피디아 크롤링 데이터
data = pd.read_csv('/content/media_data.csv', encoding = 'utf-8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18405 entries, 0 to 18404
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           18405 non-null  object 
 1   Runtime         18405 non-null  float64
 2   Release Date    18405 non-null  object 
 3   Certification   15783 non-null  object 
 4   Genres          18405 non-null  object 
 5   Origin Country  18334 non-null  object 
 6   Overview        18405 non-null  object 
 7   Director        15274 non-null  object 
 8   Cast            17855 non-null  object 
 9   Providers       16331 non-null  object 
 10  Rating Value    18405 non-null  float64
 11  Rating Count    18405 non-null  int64  
 12  Poster URL      18405 non-null  object 
 13  Backdrop URLs   18405 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 2.0+ MB


In [ ]:
# 필요한 컬럼(피처)만 선택해서 새로운 데이터셋 생성
content = data[['Title', 'Genres', 'Overview', 'Rating Value', 'Rating Count']]

print(content.head())
print(content.columns)

        Title             Genres  \
0     성냥공장 소녀           코미디, 드라마   
1     키다리 아저씨    코미디, 애니메이션, 드라마   
2  주윤발의 행운의 별  코미디, 로맨스, 로맨틱 코미디   
3        부귀병단            코미디, 액션   
4        불가사리    SF, 액션, 공포, 코미디   

                                            Overview  Rating Value  \
0  무능력하고 무표정한 얼굴의 엄마와 계부의 생활비를 위해 매일같이 성냥공장에서 기계처...           3.8   
1  고아이지만 언제나 밝은 주디와 그런 주디를 도와주는 후견인 '키다리 아저씨'의 사랑...           4.2   
2  재벌가 도련님 임보생은 재산 상속권을 조건으로 육촌 동생 진옥선과의 혼인을 강요당한...           3.2   
3  1940년대 초, 일본의 침공으로 전 중국이 혼란에 빠져있을 무렵, 중국내 일본의 ...           2.9   
4  네바다주 사막 한 가운데 있는 작은 마을에는 20명도 안되는 주민들이 서로 도우며 ...           3.3   

   Rating Count  
0          5278  
1          1623  
2           184  
3           128  
4         61592  
Index(['Title', 'Genres', 'Overview', 'Rating Value', 'Rating Count'], dtype='object')


In [ ]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18405 entries, 0 to 18404
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         18405 non-null  object 
 1   Genres        18405 non-null  object 
 2   Overview      18405 non-null  object 
 3   Rating Value  18405 non-null  float64
 4   Rating Count  18405 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 719.1+ KB


In [ ]:
# MBTI 500 데이터
mbti = pd.read_csv('/content/MBTI 500.csv')
print(mbti.info())

# 2. LaBSE 모델 로드 및 임베딩

- 사전 학습된 LaBSE 모델을 사용하여 텍스트 데이터를 고차원 벡터로 변환합니다.
- 이 임베딩 모델은 다국어 문장 임베딩을 위한 모델로, 15개 언어만 사용하도록 하여 경량화된 모델입니다.
- 고차원 벡터로 변환된 텍스트는 유사도 계산에 사용됩니다.

In [ ]:
# LaBSE 모델 로드
encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1")
preprocessor = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1")

# 텍스트를 고차원 벡터로 인코딩하는 모델 구성
def build_embedding_model():
    sentences = tf.keras.layers.Input(shape=(), dtype=tf.string, name="sentences")
    encoder_inputs = preprocessor(sentences)
    sentence_representation = encoder(encoder_inputs)["pooled_output"]
    normalized_sentence_representation = tf.nn.l2_normalize(sentence_representation, axis=-1)  # for cosine similarity
    return tf.keras.Model(sentences, normalized_sentence_representation)

embedding_model = build_embedding_model()
embedding_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sentences (InputLayer)         [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_mask': (Non  0           ['sentences[0][0]']              
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
# 텍스트 임베딩 함수
def embed_texts(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_tensors = tf.constant(batch_texts)
        batch_embeddings = embedding_model(batch_tensors).numpy()
        embeddings.extend(batch_embeddings)
    return embeddings

In [ ]:
# MBTI 임베딩
mbti_posts = mbti.groupby('type')['posts'].apply(lambda posts: ' '.join(posts)).tolist()
mbti_embeddings = embed_texts(mbti_posts)
mbti_index = mbti.groupby('type').groups.keys()
mbti_embeddings_dict = dict(zip(mbti_index, mbti_embeddings))

# 콘텐츠 줄거리 임베딩
contents_texts = content['Overview'].tolist()
contents_embeddings = embed_texts(contents_texts)
contents_index = content['Title'].tolist()
contents_embeddings_dict = dict(zip(contents_index, contents_embeddings))

In [ ]:
# 콘텐츠 임베딩 파일 저장 및 로드
with open('contents_embeddings_dict.pkl', 'wb') as f:
    pickle.dump(contents_embeddings_dict, f)

In [ ]:
# MBTI 임베딩 파일 저장 및 로드
with open('mbti_embeddings_dict.pkl', 'rb') as f:
    mbti_embeddings_dict = pickle.load(f)

In [ ]:
# 콘텐츠 임베딩 로드
with open('contents_embeddings_dict_0612.pkl', 'rb') as f:
    contents_embeddings_dict = pickle.load(f)

# 3. 콘텐츠 장르 원-핫 인코딩 / 데이터 표준화 / 콘텐츠 평점 개수 정규화

- 콘텐츠 장르를 원-핫 인코딩하여 장르 정보를 벡터화합니다.
- 원-핫 인코딩된 장르 벡터와 평점 데이터를 결합하여 학습 데이터를 생성합니다.
- 데이터 표준화를 통해 모델 학습을 용이하게 합니다.
- 대중성 점수로 사용하기 위해 콘텐츠 평점 개수를 정규화합니다.

In [ ]:
# 콘텐츠 장르 원-핫 인코딩
genres = content['Genres'].str.get_dummies(sep=', ')

# 평점 데이터 결합
cbf_model_input = np.hstack([genres.values, content['Rating Value'].values.reshape(-1, 1)])

# 데이터 표준화
cbf_scaler = StandardScaler()
cbf_model_input_scaled = cbf_scaler.fit_transform(cbf_model_input)

In [ ]:
# 콘텐츠 평점 개수를 정규화하는 함수
def normalize_popularity_score(content):
    scaler = MinMaxScaler()
    content['Normalized Popularity Score'] = scaler.fit_transform(content[['Rating Count']])
    return content

content = normalize_popularity_score(content)

<ipython-input-14-e69302caf052>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['Normalized Popularity Score'] = scaler.fit_transform(content[['Rating Count']])


# 4. 모델 학습 및 최적화

- 학습 데이터와 평가 데이터로 분리합니다.
- GBM(Gradient Boosting Machine)을 사용하여 모델을 학습합니다.
- Grid Search를 통해 최적의 하이퍼파라미터를 탐색하고, 최적의 모델을 선정합니다.

In [ ]:
# 학습 데이터와 평가 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(cbf_model_input_scaled, content['Rating Value'].values, test_size=0.2, random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.8, 1.0]
}

# 그리드 서치 설정
gbm = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=1, verbose=2)

# 그리드 서치 수행
grid_search.fit(X_train, y_train)

# 최적의 모델 출력
best_gbm = grid_search.best_estimator_
print(f'Best GBM Model: {best_gbm}')

# 검증 데이터로 모델 성능 평가
y_pred = best_gbm.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f'Validation MSE: {mse}')

# 모델 저장
joblib.dump(best_gbm, 'best_gbm_model_final.pkl')

In [ ]:
# 최적의 모델 로드
best_gbm = joblib.load('best_gbm_model_final.pkl')

# 5. 콘텐츠 추천 시스템 구현

- **유사도 계산 함수**: 두 임베딩 벡터 간의 코사인 유사도를 계산합니다.
- **MBTI 기반 추천**: 사용자의 MBTI 임베딩과 콘텐츠 임베딩 간의 유사도를 계산하여 콘텐츠를 추천합니다.
- **선호 콘텐츠 기반 추천**: 사용자가 선호하는 콘텐츠와 유사한 콘텐츠를 추천합니다.

In [ ]:
# 콘텐츠와 MBTI 유형 간의 유사도 계산 함수
def calculate_similarity(content_embedding, user_embedding):
    from numpy import dot
    from numpy.linalg import norm
    return dot(content_embedding, user_embedding) / (norm(content_embedding) * norm(user_embedding))

# 1. MBTI 임베딩 기반 추천
def recommend_contents_by_mbti(user_embedding, content_embeddings_dict, top_n=100):
    recommended_contents = []
    for content_id, embedding in content_embeddings_dict.items():
        similarity = calculate_similarity(embedding, user_embedding)
        recommended_contents.append((content_id, similarity))
    recommended_contents = sorted(recommended_contents, key=lambda x: x[1], reverse=True)
    return recommended_contents[:top_n]

# 2. 선호 콘텐츠 유사도 기반 추천
def recommend_similar_contents(preferred_contents, content_embeddings_dict, top_n=100):
    similar_contents = {}
    for content in preferred_contents:
        content_embedding = content_embeddings_dict[content]
        for other_content, embedding in content_embeddings_dict.items():
            if other_content not in preferred_contents:
                similarity = calculate_similarity(embedding, content_embedding)
                if other_content not in similar_contents:
                    similar_contents[other_content] = 0
                similar_contents[other_content] += similarity
    similar_contents = sorted(similar_contents.items(), key=lambda x: x[1], reverse=True)
    return similar_contents[:top_n]

- **결합 추천 시스템**: MBTI 유사도 점수, 선호 콘텐츠 유사도 점수, 모델 점수, 대중성 점수에 각각 가중치를 부여하여 콘텐츠를 추천합니다.

In [ ]:
# 결합 추천 시스템
def recommend_contents_combined(user_embedding, preferred_contents, content_embeddings_dict, model, content, cbf_model_input_scaled, top_n=20):
    weight_mbti = 0.3
    weight_similar = 0.4
    weight_model = 0.1
    weight_popularity = 0.2

    mbti_recommendations = recommend_contents_by_mbti(user_embedding, content_embeddings_dict, top_n=100)
    similar_content_recommendations = recommend_similar_contents(preferred_contents, content_embeddings_dict, top_n=100)

    combined_recommendations = {}

    # MBTI 유사도 점수 보정
    mbti_scores = [score for _, score in mbti_recommendations]
    mbti_std = np.std(mbti_scores)
    mbti_scores_normalized = [score / mbti_std for score in mbti_scores]

    for (content_id, _), score in zip(mbti_recommendations, mbti_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_mbti * score

    # 선호 콘텐츠 유사도 점수 보정
    similar_scores = [score for _, score in similar_content_recommendations]
    similar_std = np.std(similar_scores)
    similar_scores_normalized = [score / similar_std for score in similar_scores]

    for (content_id, _), score in zip(similar_content_recommendations, similar_scores_normalized):
        if content_id not in combined_recommendations:
            combined_recommendations[content_id] = 0
        combined_recommendations[content_id] += weight_similar * score

    # 모델 점수 보정
    content_indices = [content.index[content['Title'] == content_id][0] for content_id in combined_recommendations.keys()]
    if content_indices:
        content_features = cbf_model_input_scaled[content_indices]
        model_scores = model.predict(content_features).flatten()
        model_std = np.std(model_scores)
        model_scores_normalized = [score / model_std for score in model_scores]

        for content_id, model_score in zip(combined_recommendations.keys(), model_scores_normalized):
            combined_recommendations[content_id] += weight_model * model_score

    # 대중성 점수 보정
    popularity_scores = [content.loc[content['Title'] == content_id, 'Normalized Popularity Score'].values[0] for content_id in combined_recommendations.keys()]
    popularity_std = np.std(popularity_scores)
    popularity_scores_normalized = [score / popularity_std for score in popularity_scores]

    for content_id, popularity_score in zip(combined_recommendations.keys(), popularity_scores_normalized):
        combined_recommendations[content_id] += weight_popularity * popularity_score

    filtered_recommendations = {content_id: score for content_id, score in combined_recommendations.items()
                                if content.loc[content['Title'] == content_id, 'Rating Value'].values[0] >= 3.5}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [content_id for content_id, _ in final_recommendations[:top_n]]

- 재추천 시스템: 첫 번째 추천 목록에서 사용자가 마음에 드는(선택한) 콘텐츠를 기반으로 이전에 추천된 콘텐츠와 처음에 선호 콘텐츠로 선택한 콘텐츠를 제외하고 재추천을 수행합니다.

In [ ]:
# 재추천 시스템
def recommend_contents_combined_excluding(user_embedding, preferred_contents, previous_recommendations, initial_preferred_contents, content_embeddings_dict, model, content, cbf_model_input_scaled, top_n=20):
    weight_mbti = 0.3
    weight_similar = 0.4
    weight_model = 0.1
    weight_popularity = 0.2

    mbti_recommendations = recommend_contents_by_mbti(user_embedding, content_embeddings_dict, top_n=100)
    similar_content_recommendations = recommend_similar_contents(preferred_contents, content_embeddings_dict, top_n=100)

    combined_recommendations = {}

    all_recommendations = set([content_id for content_id, _ in mbti_recommendations] +
                              [content_id for content_id, _ in similar_content_recommendations])
    filtered_content_ids = {content_id for content_id in all_recommendations
                            if content_id not in previous_recommendations and
                            content_id not in preferred_contents}

    for content_id in filtered_content_ids:
        combined_recommendations[content_id] = 0

    # MBTI 유사도 점수 보정
    mbti_scores = [score for content_id, score in mbti_recommendations if content_id in filtered_content_ids]
    mbti_std = np.std(mbti_scores)
    mbti_scores_normalized = [score / mbti_std for score in mbti_scores]

    for (content_id, _), score in zip(mbti_recommendations, mbti_scores_normalized):
        if content_id in filtered_content_ids:
            combined_recommendations[content_id] += weight_mbti * score

    # 선호 콘텐츠 유사도 점수 보정
    similar_scores = [score for content_id, score in similar_content_recommendations if content_id in filtered_content_ids]
    similar_std = np.std(similar_scores)
    similar_scores_normalized = [score / similar_std for score in similar_scores]

    for (content_id, _), score in zip(similar_content_recommendations, similar_scores_normalized):
        if content_id in filtered_content_ids:
            combined_recommendations[content_id] += weight_similar * score

    # 모델 점수 보정
    content_indices = [content.index[content['Title'] == content_id][0] for content_id in filtered_content_ids]
    if content_indices:
        content_features = cbf_model_input_scaled[content_indices]
        model_scores = model.predict(content_features).flatten()
        model_std = np.std(model_scores)
        model_scores_normalized = [score / model_std for score in model_scores]

        for content_id, model_score in zip(filtered_content_ids, model_scores_normalized):
            combined_recommendations[content_id] += weight_model * model_score

    # 대중성 점수 보정
    popularity_scores = [content.loc[content['Title'] == content_id, 'Normalized Popularity Score'].values[0] for content_id in filtered_content_ids]
    popularity_std = np.std(popularity_scores)
    popularity_scores_normalized = [score / popularity_std for score in popularity_scores]

    for content_id, popularity_score in zip(filtered_content_ids, popularity_scores_normalized):
        combined_recommendations[content_id] += weight_popularity * popularity_score

    filtered_recommendations = {content_id: score for content_id, score in combined_recommendations.items()
                                if content.loc[content['Title'] == content_id, 'Rating Value'].values[0] >= 3.5}

    final_recommendations = sorted(filtered_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [content_id for content_id, _ in final_recommendations[:top_n]]

In [ ]:
# 사용자 입력 추천 함수
def user_input_recommendation(mbti_type, preferred_contents, content, cbf_model_input_scaled):
    user_embedding = mbti_embeddings_dict[mbti_type]

    # 최적화된 결합 추천 시스템 실행
    recommendations = recommend_contents_combined(user_embedding, preferred_contents, contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled)

    return recommendations

# 6. 추천 실행

- 예시 사용자의 입력 데이터를 사용해서 결합 추천 시스템과 재추천 시스템을 실행합니다.

In [ ]:
# 예시 사용자 입력 데이터
new_user_mbti = "INTP"
new_user_preferred_contents = ['어벤져스: 엔드게임', '어메이징 스파이더맨']

# 최적화된 결합 추천 시스템 실행
start_time = time.time()
optimized_recommendations = user_input_recommendation(new_user_mbti, new_user_preferred_contents, content, cbf_model_input_scaled)
end_time = time.time()
print("최적화된 결합 추천 콘텐츠 목록:", optimized_recommendations)
print(f"최적화된 추천 시스템 실행 시간: {end_time - start_time:.2f} 초")

최적화된 결합 추천 콘텐츠 목록: ['매트릭스', '스파이더맨: 파 프롬 홈', '어메이징 스파이더맨 2', '토르: 다크 월드', '오블리비언', '스파이더맨 3', '캡틴 아메리카: 윈터 솔져', '아이언맨 3', '스파이더맨 2', '스파이더맨', '미스 페레그린과 이상한 아이들의 집', '나비 효과', '나니아 연대기: 새벽 출정호의 항해', '종말의 세라프', '머나먼 세상속으로', '피니와 퍼브 무비: 2차원을 넘어서', '오펀: 천사의 비밀', '프린세스 스타의 모험일기', '가디언즈 오브 갤럭시 Vol. 2', '판의 미로: 오필리아와 세개의 열쇠']
최적화된 추천 시스템 실행 시간: 1.71 초


In [ ]:
# 첫 번째 추천으로부터 재추천
liked_contents_input = input("첫 번째 추천 목록에서 선호하는 콘텐츠를 쉼표로 구분하여 입력하세요: ")
liked_contents = [content.strip() for content in liked_contents_input.split(',')]

previous_recommendations = optimized_recommendations
new_preferred_contents = liked_contents # 사용자 입력을 기반으로 새로운 선호 콘텐츠 목록 생성

# 재추천 시스템 실행 시간 측정
start_time = time.time()
new_user_embedding = mbti_embeddings_dict[new_user_mbti]
second_recommendations = recommend_contents_combined_excluding(new_user_embedding, new_preferred_contents, previous_recommendations, new_user_preferred_contents, contents_embeddings_dict, best_gbm, content, cbf_model_input_scaled)
end_time = time.time()

print("재추천 콘텐츠 목록:", second_recommendations)
print(f"재추천 시스템 실행 시간: {end_time - start_time:.2f} 초")

첫 번째 추천 목록에서 선호하는 콘텐츠를 쉼표로 구분하여 입력하세요: 매트릭스, 스파이더맨: 파 프롬 홈
재추천 콘텐츠 목록: ['어벤져스', '어벤져스: 엔드게임', '매트릭스 3: 레볼루션', '월드워Z', '맨 인 블랙 3', '메이즈 러너', '전익의 시그드리파', '어벤져스: 에이지 오브 울트론', '맨 인 블랙', '드래곤볼 슈퍼', '아머드 사우루스', '혹성탈출: 반격의 서막', '트랜스포머 프라임 비스트헌터: 프레데콘 라이징', '엑스맨: 데이즈 오브 퓨처 패스트', '스파이더맨: 뉴 유니버스', '터미네이터 2: 심판의 날', '터닝메카드', '낙원추방', '취성의 가르간티아', '나소흑전기: 첫만남편']
재추천 시스템 실행 시간: 1.69 초


# 7. MVTI 추천 시스템의 특징 및 장점

**1. 하이브리드 추천 시스템**
- 이 추천 시스템은 다양한 방법을 결합하여 최적의 추천을 제공합니다.
- **MBTI 기반 추천**
  - 사용자의 MBTI 유형을 바탕으로 콘텐츠를 추천합니다. 이는 사용자의 성격 유형과 선호도에 기반하여 추천을 제공하므로, 사용자 맞춤형 추천의 정밀도를 높입니다.
  - **Cold Start 문제 해결**: 새로운 사용자의 초기 데이터가 적어도, MBTI 기반으로 개인화된 추천을 제공할 수 있어 초기에 데이터가 부족한 상황에서도 효과적입니다.

- **선호 콘텐츠 유사도 기반 추천**
  - 사용자가 이미 선호하는 콘텐츠와 유사한 특성을 가진 콘텐츠를 추천합니다.
  - 콘텐츠의 특성을 분석하여 사용자 취향에 맞춘 추천을 제공합니다.

- **모델 기반 예측**
  - 학습된 기계 학습 모델을 사용하여 콘텐츠의 평점을 예측하고, 이를 바탕으로 추천합니다.
  - 객관적인 평가 지표를 통해 콘텐츠의 품질을 고려한 추천이 가능합니다.

**2. 결합 추천 시스템**
- 여러 추천 소스를 결합하여 최적의 추천을 제공하는 시스템입니다. 각 추천 소스의 결과를 조합하여 사용자에게 최적화된 추천을 제공합니다.
- **가중치 조정**
  - MBTI 기반 추천, 선호 콘텐츠 기반 추천, 모델 기반 추천, 대중성 점수 등을 각각의 가중치를 조정하여 결합합니다.
  - 각 요소가 사용자의 취향을 얼마나 잘 반영하는지를 고려하여 가중치를 조절합니다.

- **점수 보정 및 정규화**
  - 각 추천 소스의 점수를 표준화하여 비교 가능하게 만듭니다.
  - 이로 인해 특정 소스의 점수가 과도하게 영향을 미치는 것을 방지하고, 균형 잡힌 추천이 가능해집니다.

**3. 고성능 텍스트 임베딩 모델 활용**
- **LaBSE 모델 사용**
  - LaBSE(Language-agnostic Bert Sentence Embedding) 모델을 활용하여 텍스트 데이터를 고차원 벡터로 변환합니다.
  - 이 모델은 다국어 지원이 가능하며, 높은 정확도의 임베딩을 제공합니다.
  - 콘텐츠 줄거리와 MBTI 유형별 텍스트를 효과적으로 벡터화하여 유사도를 계산할 수 있습니다.

**4. 데이터 통합 및 활용**
- **텍스트 임베딩과 메타데이터 통합**
  - 콘텐츠 줄거리(텍스트 데이터) 뿐만 아니라 장르, 평점과 같은 메타데이터도 함께 사용하여 추천을 수행합니다.
  - 단순 텍스트 유사도 기반의 추천을 넘어, 콘텐츠의 다양한 특성을 반영한 종합적인 추천을 제공합니다.

**5. 개인화된 추천**
- **사용자 성향 반영**
  - MBTI 유형을 기반으로 사용자의 성향을 반영한 콘텐츠 추천이 가능합니다.
- **사용자 선호 콘텐츠 반영**
  - 사용자가 선호하는 콘텐츠를 바탕으로 유사한 콘텐츠를 추천하여, 사용자 취향에 맞춘 개인화된 추천을 제공합니다.

**6. 동적 추천 시스템**
- **재추천 기능**
  - 이전에 추천된 콘텐츠를 제외하고 새로운 콘텐츠를 추천합니다.
  - 사용자가 이미 추천 받은 콘텐츠를 반복적으로 추천하는 것을 방지하며, 지속적으로 새로운 콘텐츠를 발견할 수 있도록 합니다.

**7. 정확한 성능 평가**
- **MSE 기반 평가**
  - 추천된 콘텐츠의 예측 평점과 실제 평점 간의 'Mean Squared Error'를 계산하여 추천 시스템의 성능을 평가합니다.
  - 추천의 정확성을 객관적으로 측정할 수 있습니다.
- **모델 성능 최적화**
  - 최적의 기계 학습 모델을 사용하여 추천의 정확성을 높입니다.
  - 모델의 성능을 지속적으로 모니터링하고 개선할 수 있습니다.

**8. 대중성 점수 반영**
- 콘텐츠의 대중성을 점수에 반영하여 인기 있는 콘텐츠를 추천합니다.
- **Normalized Popularity Score**
  - 콘텐츠의 인기도를 정규화하여 점수에 반영합니다.
  - 인기가 많은 콘텐츠를 추천하여, 사용자들이 선호할 가능성이 높은 콘텐츠를 제공합니다.

**9. 확장 가능성**
- **다양한 데이터 소스 통합**
  - 현재 추천 시스템은 MBTI 데이터와 콘텐츠 데이터에 기반하지만, 사용자 행동 데이터나 소셜 미디어 데이터 등의 추가적인 데이터 소스를 통합하여 더 정교한 추천 시스템으로 확장할 수 있습니다.

- **다국어 지원**
  - LaBSE 모델의 다국어 지원을 통해 다양한 언어 콘텐츠를 추천할 수 있습니다.
  - 이는 글로벌 사용자들에게도 유용한 추천 시스템을 제공할 수 있게 합니다.

**10. 결론**
- 이 추천 시스템은 다양한 추천 기법을 결합하여 사용자 맞춤형 추천의 정밀도와 다양성을 높입니다. 고성능 임베딩 모델과 다양한 데이터 소스를 통합하여, 사용자에게 가장 적합한 콘텐츠를 제공하며, 지속적인 성능 평가와 개선을 통해 추천의 정확성을 유지합니다. 또한, 확장 가능성과 다국어 지원을 통해 글로벌 사용자에게도 효율적인 추천 시스템을 제공합니다.

